# DB Performance exercises

In [17]:
%load_ext sql
%sql postgresql+psycopg2://localhost:5432/db_performance
import timeit
from sqlalchemy import create_engine
import pandas as pd

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


- loading modules 

In [18]:
%%sql
SELECT * FROM students;

 * postgresql+psycopg2://localhost:5432/db_performance
5 rows affected.


id,first_name,last_name,age,course_id
1,John,Doe,21,1
2,Jane,Smith,22,2
3,Mark,Twain,20,1
4,Emily,Dickinson,22,3
5,Louisa,Alcott,21,2


- looking at all the rows in the students table

In [24]:
%%sql 
SELECT * FROM courses;

 * postgresql+psycopg2://localhost:5432/db_performance
3 rows affected.


id,name
1,Mathematics
2,Physics
3,Chemistry


- looking at pathetic course table

In [19]:
%%sql
EXPLAIN ANALYZE SELECT * FROM students;

 * postgresql+psycopg2://localhost:5432/db_performance
3 rows affected.


QUERY PLAN
Seq Scan on students (cost=0.00..10.70 rows=70 width=1044) (actual time=0.008..0.010 rows=5 loops=1)
Planning Time: 0.064 ms
Execution Time: 0.020 ms


- above shows breakdown on sequential table scan/retreival
- time for query execution planning
- time for execution

==================================================================================================
## Exercise - retreive info on students under 25 and measure execution time.

In [20]:
%%sql
SELECT * FROM students WHERE age<25;

 * postgresql+psycopg2://localhost:5432/db_performance
5 rows affected.


id,first_name,last_name,age,course_id
1,John,Doe,21,1
2,Jane,Smith,22,2
3,Mark,Twain,20,1
4,Emily,Dickinson,22,3
5,Louisa,Alcott,21,2


- first ensuring we have the correct sample (this query still returns all records)

In [21]:
%%sql
EXPLAIN ANALYZE SELECT * FROM students WHERE age<25;

 * postgresql+psycopg2://localhost:5432/db_performance
4 rows affected.


QUERY PLAN
Seq Scan on students (cost=0.00..10.88 rows=23 width=1044) (actual time=0.006..0.007 rows=5 loops=1)
Filter: (age < 25)
Planning Time: 0.033 ms
Execution Time: 0.016 ms


- Using EXPLAIN ANALYZE we can see that the inclusion of the WHERE statement has increased the measured planning time but reduced execution time in EXPLAIN ANALYZE, it also decreased the sequential scan time.
- Total time should be calculated by suming planning time and execution time

> with WHERE =(0.033+0.016) = 0.049     ////////////     without = (0.064+0.02) = 0.084
- here we can see that suprisingly the WHERE statement decreased the total time even when all records are still being returned

- ALso repeating the exercise wields varying results (perhaps this would be more consitent with a larger sample size) 


==================================================================================================
## Creating and measuring indexing impact

In [23]:
%%sql
CREATE INDEX idx_students_course_id ON students (course_id);   

 * postgresql+psycopg2://localhost:5432/db_performance
Done.


[]

- Above we index the students table with the course_id value
- next exercise - SELECT query that retrieves all students along with their course names and measure the changes in execution time using the EXPLAIN ANALYZE command in PostgreSQL.

In [29]:
%%sql
SELECT students.first_name, students.last_name, courses.name
FROM students JOIN courses ON students.course_id = courses.id

 * postgresql+psycopg2://localhost:5432/db_performance
5 rows affected.


first_name,last_name,name
Mark,Twain,Mathematics
John,Doe,Mathematics
Louisa,Alcott,Physics
Jane,Smith,Physics
Emily,Dickinson,Chemistry


- Here we perform the required query first to ensure we are returning the correct data

In [30]:
%%sql
EXPLAIN ANALYZE
SELECT students.first_name, students.last_name, courses.name
FROM students JOIN courses ON students.course_id = courses.id

 * postgresql+psycopg2://localhost:5432/db_performance
8 rows affected.


QUERY PLAN
Hash Join (cost=1.11..13.09 rows=5 width=1548) (actual time=0.118..0.121 rows=5 loops=1)
Hash Cond: (courses.id = students.course_id)
-> Seq Scan on courses (cost=0.00..11.40 rows=140 width=520) (actual time=0.024..0.024 rows=3 loops=1)
-> Hash (cost=1.05..1.05 rows=5 width=1036) (actual time=0.064..0.064 rows=5 loops=1)
Buckets: 1024 Batches: 1 Memory Usage: 9kB
-> Seq Scan on students (cost=0.00..1.05 rows=5 width=1036) (actual time=0.010..0.012 rows=5 loops=1)
Planning Time: 0.177 ms
Execution Time: 0.409 ms


==================================================================================================
## Write a query to retrieve students from a specific course with an age greater than 20.
- Measure the execution time using the EXPLAIN ANALYZE command.

In [31]:
%%sql
SELECT *
FROM students
WHERE course_id = 2 and age > 20;

 * postgresql+psycopg2://localhost:5432/db_performance
2 rows affected.


id,first_name,last_name,age,course_id
2,Jane,Smith,22,2
5,Louisa,Alcott,21,2


- first creating the query to ensure we're returning the correct results.

In [32]:
%%sql
EXPLAIN ANALYZE
SELECT *
FROM students
WHERE course_id = 2 and age > 20;

 * postgresql+psycopg2://localhost:5432/db_performance
5 rows affected.


QUERY PLAN
Seq Scan on students (cost=0.00..1.07 rows=1 width=1044) (actual time=0.027..0.029 rows=2 loops=1)
Filter: ((age > 20) AND (course_id = 2))
Rows Removed by Filter: 3
Planning Time: 0.111 ms
Execution Time: 0.291 ms


- Here we use the EXPLAIN ANALYZE method to get an understanding on processing time for the query.
- In this case the query took ~0.4ms which is slower than our original queries but faster than the joint table query.

==================================================================================================
# COMPARE TWO QUERIES USING EXPLAIN ANALYZE

In [33]:
%%sql
SELECT c.id, c.name, (SELECT AVG(age) FROM students WHERE course_id = c.id) AS average_age
FROM courses c;

 * postgresql+psycopg2://localhost:5432/db_performance
3 rows affected.


id,name,average_age
1,Mathematics,20.5000000000000000
2,Physics,21.5000000000000000
3,Chemistry,22.0000000000000000


- original query attaining an average age for students on each course

In [34]:
%%sql
EXPLAIN ANALYZE
SELECT c.id, c.name, (SELECT AVG(age) FROM students WHERE course_id = c.id) AS average_age
FROM courses c;

 * postgresql+psycopg2://localhost:5432/db_performance
8 rows affected.


QUERY PLAN
Seq Scan on courses c (cost=0.00..162.25 rows=140 width=552) (actual time=0.056..0.065 rows=3 loops=1)
SubPlan 1
-> Aggregate (cost=1.07..1.08 rows=1 width=32) (actual time=0.010..0.010 rows=1 loops=3)
-> Seq Scan on students (cost=0.00..1.06 rows=1 width=4) (actual time=0.004..0.004 rows=2 loops=3)
Filter: (course_id = c.id)
Rows Removed by Filter: 3
Planning Time: 0.146 ms
Execution Time: 0.120 ms


- Initial results from EXPLAIN ANALYZE on the previous code block query.
- Total time = (0.146+0.120) = 0.266ms

In [41]:
%%sql
SELECT courses.id, courses.name, AVG(students.age) AS average_age
FROM courses
JOIN students ON courses.id = students.course_id
GROUP BY courses.id, courses.name;


 * postgresql+psycopg2://localhost:5432/db_performance
3 rows affected.


id,name,average_age
1,Mathematics,20.5000000000000000
2,Physics,21.5000000000000000
3,Chemistry,22.0000000000000000


- creating the query to find average ages of students per course while using JOIN and GROUP BY methods

In [42]:
%%sql
EXPLAIN ANALYZE
SELECT courses.id, courses.name, AVG(students.age) AS average_age
FROM courses
JOIN students ON courses.id = students.course_id
GROUP BY courses.id, courses.name;

 * postgresql+psycopg2://localhost:5432/db_performance
13 rows affected.


QUERY PLAN
GroupAggregate (cost=13.15..13.25 rows=5 width=552) (actual time=0.086..0.090 rows=3 loops=1)
Group Key: courses.id
-> Sort (cost=13.15..13.16 rows=5 width=524) (actual time=0.077..0.078 rows=5 loops=1)
Sort Key: courses.id
Sort Method: quicksort Memory: 25kB
-> Hash Join (cost=1.11..13.09 rows=5 width=524) (actual time=0.055..0.058 rows=5 loops=1)
Hash Cond: (courses.id = students.course_id)
-> Seq Scan on courses (cost=0.00..11.40 rows=140 width=520) (actual time=0.018..0.019 rows=3 loops=1)
-> Hash (cost=1.05..1.05 rows=5 width=8) (actual time=0.023..0.024 rows=5 loops=1)
Buckets: 1024 Batches: 1 Memory Usage: 9kB


- total time = (0.284 + 0.150) = 0.434
- after using EXPLAIN ANALYZE we can differ that this method has increased the total time taken by (0.434 - 0.266) = 0.1679
- I'm unsure if its accurate for continuous attempts but we can see even though the execution time is similar between both. The planning time takes alot longer with the grouping and joining tables query
- Is this even the intended exercise result ? More legible querries result in poorer exicution times ? 
- obviously not enough data to defer real conclusions.

- Apparently these results are contrary to the expected outcome - this maybe due to a host of reasons - but i beleive the results would have been more correlated to the hypotosis if a larger dataset was used.